In [25]:
# imports
import sys
import openmm
import openmm.app as app
import openmm.unit as unit

import numpy as np
import mdtraj

In [26]:
# load files
psf = app.CharmmPsfFile('init.psf')
pdbfile = app.PDBFile('init.pdb')
params = app.CharmmParameterSet('toppar/top_all36_prot.rtf', 'toppar/par_all36m_prot.prm', 'toppar/toppar_water_ions.str')
positions = pdbfile.getPositions(asNumpy=True)

In [27]:
# system
psf.setBox(39.0*unit.angstroms, 39.0*unit.angstroms, 39.0*unit.angstroms)

system = psf.createSystem(params,
    nonbondedMethod=app.PME, 
    nonbondedCutoff=1.2*unit.nanometers,
    implicitSolvent=None,
    constraints=app.HBonds,
    rigidWater=True, 
    verbose=True, 
    ewaldErrorTolerance=0.0005)

barostat = openmm.MonteCarloBarostat(1.0*unit.bar, 300.0*unit.kelvin)
system.addForce(barostat)

Adding particles...
Adding constraints...
Adding lonepairs...
Adding bonds...
Adding angles...
    Number of bond constraints: 3685
    Number of angle constraints: 1812
Adding Urey-Bradley terms
Adding torsions...
Adding impropers...
Adding CMAP coupled torsions...
Adding nonbonded interactions...
Build exclusion list...
    Number of 1-2 pairs: 3765
    Number of 1-3 pairs: 2061
    Number of 1-4 pairs: 353


8

In [28]:
# integrator
integrator = openmm.LangevinIntegrator(300.0*unit.kelvin,   # Temperature of head bath
                                       1.0/unit.picosecond, # Friction coefficient
                                       0.002*unit.picoseconds) # Time step

In [29]:
# platform
# platform = openmm.Platform.getPlatformByName('CPU')
platform = openmm.Platform.getPlatformByName('CUDA')
# prop = dict(CudaPrecision='mixed')

In [30]:
# Build simulation context
simulation = app.Simulation(psf.topology, system, integrator, platform)
#simulation = app.Simulation(psf.topology, system, integrator, platform, prop)
simulation.context.setPositions(positions)

In [31]:
# initial system energy
print("\ninitial system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(unit.kilocalories_per_mole))


initial system energy
-21573.471534575205 kcal/mol


In [32]:
simulation.minimizeEnergy(maxIterations=1000)
print("\nafter minimization")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(unit.kilocalories_per_mole))


after minimization
-22230.020291745375 kcal/mol


In [33]:
def np_to_mm(arr: np.ndarray, unit: openmm.unit=unit.angstrom):
    wrapped_val = openmm.unit.quantity.Quantity(arr, unit)
    return wrapped_val

In [35]:
nmr_pdbfile = app.PDBFile('1uao_nmr.pdb')
nmr_positions = nmr_pdbfile.getPositions(asNumpy=True)

positions_nm = simulation.context.getState(getPositions=True).getPositions()
positions_nm[:len(nmr_positions)] = nmr_positions
simulation.context.setPositions(positions_nm)

In [36]:
# initial system energy
print("\ninitial system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(unit.kilocalories_per_mole))


initial system energy
8048243550.841324 kcal/mol


In [37]:
simulation.minimizeEnergy(maxIterations=1000)
print("\nafter minimization")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(unit.kilocalories_per_mole))


after minimization
-22289.891228647863 kcal/mol
